In [198]:
import numpy as np
from numpy.lib import stride_tricks

In [199]:
def mask_to_int(mask):
    to_remove = '[] '
    s = str(mask.ravel())
    for c in to_remove:
        s = s.replace(c, '')
    return eval('0b{}'.format(s))

In [200]:
def gen_int_masks(MASKS, h=6, w=5):
    result = {}
    for r, row in enumerate(MASKS):
        for c, mask in enumerate(row):
            all_moves = mask_to_int(mask)
            result[1 << 29-(5*r+c)] = [1 << i for i in range(29,-1,-1) if (1 << i & all_moves) and (i != 29-(5*r+c))]
    return result

In [201]:
def print_bits(masks):
    for anchor, moves in masks.items():
        print('Anchor: {:030b}'.format(anchor))
        print(' Masks:')
        for move in moves:
            print('        {:030b}'.format(move))
        print('-'*40, end='\n\n')

In [202]:
def raw_heuristic_to_b_w_cpp(w_var_name, b_var_name, s, queen_hueristic=None):
    w_result = ''
    b_result = ''
    
    white_raw = np.array(s.split())
    if queen_hueristic:
        white_queen_raw = np.array(queen_hueristic.split())
    
    ppromo = (1 << 30)
    
    w_result += 'static std::unordered_map<unsigned int, int> {} = {{\n'.format(w_var_name)
    
    if 'pawn' in w_var_name:
        for i in range(29, -1, -1):
            w_result += '  {{{}, {}}},\n'.format(((1 << i) | ppromo), np.flip(white_queen_raw, axis=0)[i])
    for i in range(29, -1, -1):
        w_result += '  {{{}, {}}},\n'.format((1 << i), np.flip(white_raw, axis=0)[i])
    w_result += '  {0, 0},\n'
    w_result += '};\n\n'
    
    b_result += 'static std::unordered_map<unsigned int, int> {} = {{\n'.format(b_var_name)
    
    if 'pawn' in w_var_name:
        for i in range(29, -1, -1):
            b_result += '  {{{}, {}}},\n'.format(((1 << i) | ppromo), white_queen_raw[i])
    
    for i in range(29, -1, -1):
        b_result += '  {{{}, {}}},\n'.format((1 << i), white_raw[i])
    b_result += '  {0, 0},\n'
    b_result += '};\n\n'
    
    return w_result, b_result

In [203]:
BOARD_HEIGHT = 6
BOARD_WIDTH = 5
MASK_HEIGHT = 2 * BOARD_HEIGHT - 1
MASK_WIDTH = 2 * BOARD_WIDTH - 1
print('MASK_HEIGHT: {}\n'
      ' MASK_WIDTH: {}'.format(MASK_HEIGHT, MASK_WIDTH))
MASK_ANCHOR = (MASK_HEIGHT // 2, MASK_WIDTH // 2)
print(MASK_ANCHOR)

MASK_HEIGHT: 11
 MASK_WIDTH: 9
(5, 4)


In [204]:
BASE_MASK = np.zeros((MASK_HEIGHT, MASK_WIDTH), np.int32)

In [205]:
FULL_NS_MASK = BASE_MASK.copy()
FULL_NS_MASK[:, MASK_ANCHOR[1]] += 1
FULL_EW_MASK = BASE_MASK.copy()
FULL_EW_MASK[MASK_ANCHOR[0], :] += 1
FULL_NS_MASK, FULL_EW_MASK

(array([[0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0]], dtype=int32),
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32))

In [206]:
FULL_DIAGS_MASK = (np.eye(min(MASK_HEIGHT, MASK_WIDTH), dtype=np.int) | np.fliplr(np.eye(min(MASK_HEIGHT, MASK_WIDTH), dtype=np.int)))
tb_pad = (MASK_HEIGHT - FULL_DIAGS_MASK.shape[0]) // 2
lr_pad = (MASK_WIDTH - FULL_DIAGS_MASK.shape[1]) // 2
FULL_DIAGS_MASK = np.lib.pad(FULL_DIAGS_MASK, ((tb_pad, tb_pad),(lr_pad, lr_pad)), 'constant')
print(FULL_DIAGS_MASK, FULL_DIAGS_MASK.shape)

[[0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 1 0 0]
 [0 0 0 1 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 1 0 0 0]
 [0 0 1 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0]] (11, 9)


In [207]:
ROOK = FULL_NS_MASK | FULL_EW_MASK
print(ROOK, ROOK.shape)

[[0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [1 1 1 1 1 1 1 1 1]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]] (11, 9)


In [208]:
ROOK_MASKS = np.rot90(stride_tricks.as_strided(ROOK, shape=(6, 5, 6, 5), strides=ROOK.strides + ROOK.strides), k=2)
ROOK_MASKS

array([[[[1, 1, 1, 1, 1],
         [1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0]],

        [[1, 1, 1, 1, 1],
         [0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0]],

        [[1, 1, 1, 1, 1],
         [0, 0, 1, 0, 0],
         [0, 0, 1, 0, 0],
         [0, 0, 1, 0, 0],
         [0, 0, 1, 0, 0],
         [0, 0, 1, 0, 0]],

        [[1, 1, 1, 1, 1],
         [0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0]],

        [[1, 1, 1, 1, 1],
         [0, 0, 0, 0, 1],
         [0, 0, 0, 0, 1],
         [0, 0, 0, 0, 1],
         [0, 0, 0, 0, 1],
         [0, 0, 0, 0, 1]]],


       [[[1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1],
         [1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0]],

        [[0, 1, 0, 0, 0],
         [1, 1, 1, 1, 1]

In [209]:
QUEEN = FULL_DIAGS_MASK | ROOK
print(QUEEN, QUEEN.shape)

[[0 0 0 0 1 0 0 0 0]
 [1 0 0 0 1 0 0 0 1]
 [0 1 0 0 1 0 0 1 0]
 [0 0 1 0 1 0 1 0 0]
 [0 0 0 1 1 1 0 0 0]
 [1 1 1 1 1 1 1 1 1]
 [0 0 0 1 1 1 0 0 0]
 [0 0 1 0 1 0 1 0 0]
 [0 1 0 0 1 0 0 1 0]
 [1 0 0 0 1 0 0 0 1]
 [0 0 0 0 1 0 0 0 0]] (11, 9)


In [210]:
QUEEN_MASKS = np.rot90(stride_tricks.as_strided(QUEEN, shape=(6, 5, 6, 5), strides=QUEEN.strides + QUEEN.strides), k=2)
QUEEN_MASKS

array([[[[1, 1, 1, 1, 1],
         [1, 1, 0, 0, 0],
         [1, 0, 1, 0, 0],
         [1, 0, 0, 1, 0],
         [1, 0, 0, 0, 1],
         [1, 0, 0, 0, 0]],

        [[1, 1, 1, 1, 1],
         [1, 1, 1, 0, 0],
         [0, 1, 0, 1, 0],
         [0, 1, 0, 0, 1],
         [0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0]],

        [[1, 1, 1, 1, 1],
         [0, 1, 1, 1, 0],
         [1, 0, 1, 0, 1],
         [0, 0, 1, 0, 0],
         [0, 0, 1, 0, 0],
         [0, 0, 1, 0, 0]],

        [[1, 1, 1, 1, 1],
         [0, 0, 1, 1, 1],
         [0, 1, 0, 1, 0],
         [1, 0, 0, 1, 0],
         [0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0]],

        [[1, 1, 1, 1, 1],
         [0, 0, 0, 1, 1],
         [0, 0, 1, 0, 1],
         [0, 1, 0, 0, 1],
         [1, 0, 0, 0, 1],
         [0, 0, 0, 0, 1]]],


       [[[1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1],
         [1, 1, 0, 0, 0],
         [1, 0, 1, 0, 0],
         [1, 0, 0, 1, 0],
         [1, 0, 0, 0, 1]],

        [[1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1]

In [211]:
KING = np.lib.pad(np.ones((3,3), dtype=np.int), (((MASK_HEIGHT-3)//2, (MASK_HEIGHT-3)//2),((MASK_WIDTH-3)//2, (MASK_WIDTH-3)//2)), 'constant')
print(KING, KING.shape)

[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 1 0 0 0]
 [0 0 0 1 1 1 0 0 0]
 [0 0 0 1 1 1 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]] (11, 9)


In [212]:
KING_MASKS = np.rot90(stride_tricks.as_strided(KING, shape=(6, 5, 6, 5), strides=KING.strides + KING.strides), k=2)
KING_MASKS

array([[[[1, 1, 0, 0, 0],
         [1, 1, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[1, 1, 1, 0, 0],
         [1, 1, 1, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 1, 1, 1, 0],
         [0, 1, 1, 1, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 1, 1, 1],
         [0, 0, 1, 1, 1],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 1],
         [0, 0, 0, 1, 1],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[1, 1, 0, 0, 0],
         [1, 1, 0, 0, 0],
         [1, 1, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[1, 1, 1, 0, 0],
         [1, 1, 1, 0, 0]

In [213]:
BISHOP = {
    'ATTACK': FULL_DIAGS_MASK,
    'MOVE'  : KING | FULL_DIAGS_MASK
}
print('ATTACK')
print(BISHOP['ATTACK'])
print('MOVE')
print(BISHOP['MOVE'])

ATTACK
[[0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 1 0 0]
 [0 0 0 1 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 1 0 0 0]
 [0 0 1 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0]]
MOVE
[[0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 1 0 0]
 [0 0 0 1 1 1 0 0 0]
 [0 0 0 1 1 1 0 0 0]
 [0 0 0 1 1 1 0 0 0]
 [0 0 1 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0]]


In [214]:
BISHOP_ATTACK_MASKS = np.rot90(stride_tricks.as_strided(BISHOP['ATTACK'], shape=(6, 5, 6, 5), strides=BISHOP['ATTACK'].strides + BISHOP['MOVE'].strides), k=2)
BISHOP_ATTACK_MASKS

array([[[[1, 0, 0, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 0, 1, 0, 0],
         [0, 0, 0, 1, 0],
         [0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0]],

        [[0, 1, 0, 0, 0],
         [1, 0, 1, 0, 0],
         [0, 0, 0, 1, 0],
         [0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 1, 0, 0],
         [0, 1, 0, 1, 0],
         [1, 0, 0, 0, 1],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 0],
         [0, 0, 1, 0, 1],
         [0, 1, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1],
         [0, 0, 0, 1, 0],
         [0, 0, 1, 0, 0],
         [0, 1, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[0, 1, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 0, 1, 0, 0],
         [0, 0, 0, 1, 0],
         [0, 0, 0, 0, 1]],

        [[1, 0, 1, 0, 0],
         [0, 1, 0, 0, 0]

In [215]:
BISHOP_MOVE_MASKS = np.rot90(stride_tricks.as_strided(BISHOP['MOVE'], shape=(6, 5, 6, 5), strides=BISHOP['MOVE'].strides + BISHOP['MOVE'].strides), k=2)
BISHOP_MOVE_MASKS

array([[[[1, 1, 0, 0, 0],
         [1, 1, 0, 0, 0],
         [0, 0, 1, 0, 0],
         [0, 0, 0, 1, 0],
         [0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0]],

        [[1, 1, 1, 0, 0],
         [1, 1, 1, 0, 0],
         [0, 0, 0, 1, 0],
         [0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 1, 1, 1, 0],
         [0, 1, 1, 1, 0],
         [1, 0, 0, 0, 1],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 1, 1, 1],
         [0, 0, 1, 1, 1],
         [0, 1, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 1],
         [0, 0, 0, 1, 1],
         [0, 0, 1, 0, 0],
         [0, 1, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[1, 1, 0, 0, 0],
         [1, 1, 0, 0, 0],
         [1, 1, 0, 0, 0],
         [0, 0, 1, 0, 0],
         [0, 0, 0, 1, 0],
         [0, 0, 0, 0, 1]],

        [[1, 1, 1, 0, 0],
         [1, 1, 1, 0, 0]

In [216]:
BLACK_PAWN = {
    'ATTACK': np.lib.pad(np.array([[0,1,0],[1,0,1]], dtype=np.int), (((MASK_HEIGHT - 2)//2 + 1,4),(3,3)), 'constant'),
    'MOVE'  : np.lib.pad(np.ones((2,1), dtype=np.int), (((MASK_HEIGHT - 2)//2 + 1,4),(4,4)), 'constant')
}
print('ATTACK')
print(BLACK_PAWN['ATTACK'])
print('MOVE')
print(BLACK_PAWN['MOVE'])

ATTACK
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
MOVE
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]


In [217]:
B_PAWN_ATTACK_MASKS = np.rot90(stride_tricks.as_strided(BLACK_PAWN['ATTACK'], shape=(6, 5, 6, 5), strides=BLACK_PAWN['ATTACK'].strides + BLACK_PAWN['MOVE'].strides), k=2)
B_PAWN_ATTACK_MASKS

array([[[[1, 0, 0, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 1, 0, 0, 0],
         [1, 0, 1, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 1, 0, 0],
         [0, 1, 0, 1, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 0],
         [0, 0, 1, 0, 1],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1],
         [0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0]

In [218]:
B_PAWN_MOVE_MASKS = np.rot90(stride_tricks.as_strided(BLACK_PAWN['MOVE'], shape=(6, 5, 6, 5), strides=BLACK_PAWN['MOVE'].strides + BLACK_PAWN['MOVE'].strides), k=2)
B_PAWN_MOVE_MASKS

array([[[[1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 1, 0, 0],
         [0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1],
         [0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0]

In [219]:
WHITE_PAWN = {
    'ATTACK': np.flipud(BLACK_PAWN['ATTACK']),
    'MOVE'  : np.flipud(BLACK_PAWN['MOVE'])
}
print('ATTACK')
print(WHITE_PAWN['ATTACK'])
print('MOVE')
print(WHITE_PAWN['MOVE'])

ATTACK
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
MOVE
[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]


In [220]:
W_PAWN_MOVE_MASKS = np.rot90(stride_tricks.as_strided(WHITE_PAWN['MOVE'], shape=(6, 5, 6, 5), strides=WHITE_PAWN['MOVE'].strides + WHITE_PAWN['MOVE'].strides), k=2)
W_PAWN_MOVE_MASKS

array([[[[1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0]

In [221]:
W_PAWN_ATTACK_MASKS = np.rot90(stride_tricks.as_strided(WHITE_PAWN['ATTACK'], shape=(6, 5, 6, 5), strides=WHITE_PAWN['ATTACK'].strides + WHITE_PAWN['ATTACK'].strides), k=2)
W_PAWN_ATTACK_MASKS

array([[[[1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[0, 1, 0, 0, 0],
         [1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[1, 0, 1, 0, 0],
         [0, 1, 0, 0, 0]

In [222]:
KNIGHT = np.lib.pad(np.array(
    [[0,1,0,1,0],
     [1,0,0,0,1],
     [0,0,1,0,0],
     [1,0,0,0,1],
     [0,1,0,1,0]]), ((3,3),(2,2)), 'constant')
print(KNIGHT)

[[0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 1 0 0 0]
 [0 0 1 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 1 0 0]
 [0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]


In [223]:
KNIGHT_MASKS = np.rot90(stride_tricks.as_strided(KNIGHT, shape=(6, 5, 6, 5), strides=KNIGHT.strides + KNIGHT.strides), k=2)
KNIGHT_MASKS

array([[[[1, 0, 0, 0, 0],
         [0, 0, 1, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0],
         [1, 0, 1, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 1, 0, 0],
         [1, 0, 0, 0, 1],
         [0, 1, 0, 1, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 0],
         [0, 1, 0, 0, 0],
         [0, 0, 1, 0, 1],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 1],
         [0, 0, 1, 0, 0],
         [0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]]],


       [[[0, 0, 1, 0, 0],
         [1, 0, 0, 0, 0],
         [0, 0, 1, 0, 0],
         [0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0]],

        [[0, 0, 0, 1, 0],
         [0, 1, 0, 0, 0]

In [224]:
KNIGHT_MASKS[(3,2)].ravel()

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 0])

In [225]:
import pandas as pd
n_masks = gen_int_masks(KNIGHT_MASKS)
print(n_masks)
print_bits(n_masks)

{536870912: [4194304, 262144], 268435456: [2097152, 524288, 131072], 134217728: [16777216, 1048576, 262144, 65536], 67108864: [8388608, 131072, 32768], 33554432: [4194304, 65536], 16777216: [134217728, 131072, 8192], 8388608: [67108864, 65536, 16384, 4096], 4194304: [536870912, 33554432, 524288, 32768, 8192, 2048], 2097152: [268435456, 262144, 4096, 1024], 1048576: [134217728, 131072, 2048], 524288: [268435456, 4194304, 4096, 256], 262144: [536870912, 134217728, 2097152, 2048, 512, 128], 131072: [268435456, 67108864, 16777216, 1048576, 16384, 1024, 256, 64], 65536: [134217728, 33554432, 8388608, 8192, 128, 32], 32768: [67108864, 4194304, 4096, 64], 16384: [8388608, 131072, 128, 8], 8192: [16777216, 4194304, 65536, 64, 16, 4], 4096: [8388608, 2097152, 524288, 32768, 512, 32, 8, 2], 2048: [4194304, 1048576, 262144, 256, 4, 1], 1024: [2097152, 131072, 128, 2], 512: [262144, 4096, 4], 256: [524288, 131072, 2048, 2], 128: [262144, 65536, 16384, 1024, 16, 1], 64: [131072, 32768, 8192, 8], 32

In [226]:
w_pm_masks = gen_int_masks(W_PAWN_MOVE_MASKS)
print(w_pm_masks)
print_bits(w_pm_masks)

{536870912: [], 268435456: [], 134217728: [], 67108864: [], 33554432: [], 16777216: [536870912], 8388608: [268435456], 4194304: [134217728], 2097152: [67108864], 1048576: [33554432], 524288: [16777216], 262144: [8388608], 131072: [4194304], 65536: [2097152], 32768: [1048576], 16384: [524288], 8192: [262144], 4096: [131072], 2048: [65536], 1024: [32768], 512: [16384], 256: [8192], 128: [4096], 64: [2048], 32: [1024], 16: [512], 8: [256], 4: [128], 2: [64], 1: [32]}
Anchor: 100000000000000000000000000000
 Masks:
----------------------------------------

Anchor: 010000000000000000000000000000
 Masks:
----------------------------------------

Anchor: 001000000000000000000000000000
 Masks:
----------------------------------------

Anchor: 000100000000000000000000000000
 Masks:
----------------------------------------

Anchor: 000010000000000000000000000000
 Masks:
----------------------------------------

Anchor: 000001000000000000000000000000
 Masks:
        100000000000000000000000000000


In [227]:
w_pa_masks = gen_int_masks(W_PAWN_ATTACK_MASKS)
print(w_pa_masks)
print_bits(w_pa_masks)

{536870912: [], 268435456: [], 134217728: [], 67108864: [], 33554432: [], 16777216: [268435456], 8388608: [536870912, 134217728], 4194304: [268435456, 67108864], 2097152: [134217728, 33554432], 1048576: [67108864], 524288: [8388608], 262144: [16777216, 4194304], 131072: [8388608, 2097152], 65536: [4194304, 1048576], 32768: [2097152], 16384: [262144], 8192: [524288, 131072], 4096: [262144, 65536], 2048: [131072, 32768], 1024: [65536], 512: [8192], 256: [16384, 4096], 128: [8192, 2048], 64: [4096, 1024], 32: [2048], 16: [256], 8: [512, 128], 4: [256, 64], 2: [128, 32], 1: [64]}
Anchor: 100000000000000000000000000000
 Masks:
----------------------------------------

Anchor: 010000000000000000000000000000
 Masks:
----------------------------------------

Anchor: 001000000000000000000000000000
 Masks:
----------------------------------------

Anchor: 000100000000000000000000000000
 Masks:
----------------------------------------

Anchor: 000010000000000000000000000000
 Masks:
--------------

In [228]:
b_pm_masks = gen_int_masks(B_PAWN_MOVE_MASKS)
print(b_pm_masks)
print_bits(b_pm_masks)

{536870912: [16777216], 268435456: [8388608], 134217728: [4194304], 67108864: [2097152], 33554432: [1048576], 16777216: [524288], 8388608: [262144], 4194304: [131072], 2097152: [65536], 1048576: [32768], 524288: [16384], 262144: [8192], 131072: [4096], 65536: [2048], 32768: [1024], 16384: [512], 8192: [256], 4096: [128], 2048: [64], 1024: [32], 512: [16], 256: [8], 128: [4], 64: [2], 32: [1], 16: [], 8: [], 4: [], 2: [], 1: []}
Anchor: 100000000000000000000000000000
 Masks:
        000001000000000000000000000000
----------------------------------------

Anchor: 010000000000000000000000000000
 Masks:
        000000100000000000000000000000
----------------------------------------

Anchor: 001000000000000000000000000000
 Masks:
        000000010000000000000000000000
----------------------------------------

Anchor: 000100000000000000000000000000
 Masks:
        000000001000000000000000000000
----------------------------------------

Anchor: 000010000000000000000000000000
 Masks:
        0

In [229]:
b_pa_masks = gen_int_masks(B_PAWN_ATTACK_MASKS)
print(b_pa_masks)
print_bits(b_pa_masks)

{536870912: [8388608], 268435456: [16777216, 4194304], 134217728: [8388608, 2097152], 67108864: [4194304, 1048576], 33554432: [2097152], 16777216: [262144], 8388608: [524288, 131072], 4194304: [262144, 65536], 2097152: [131072, 32768], 1048576: [65536], 524288: [8192], 262144: [16384, 4096], 131072: [8192, 2048], 65536: [4096, 1024], 32768: [2048], 16384: [256], 8192: [512, 128], 4096: [256, 64], 2048: [128, 32], 1024: [64], 512: [8], 256: [16, 4], 128: [8, 2], 64: [4, 1], 32: [2], 16: [], 8: [], 4: [], 2: [], 1: []}
Anchor: 100000000000000000000000000000
 Masks:
        000000100000000000000000000000
----------------------------------------

Anchor: 010000000000000000000000000000
 Masks:
        000001000000000000000000000000
        000000010000000000000000000000
----------------------------------------

Anchor: 001000000000000000000000000000
 Masks:
        000000100000000000000000000000
        000000001000000000000000000000
----------------------------------------

Anchor: 0001000

In [230]:
b_attack_masks = gen_int_masks(BISHOP_ATTACK_MASKS)
print(b_attack_masks)
print_bits(b_attack_masks)

{536870912: [8388608, 131072, 2048, 32], 268435456: [16777216, 4194304, 65536, 1024], 134217728: [8388608, 2097152, 524288, 32768], 67108864: [4194304, 1048576, 262144, 16384], 33554432: [2097152, 131072, 8192, 512], 16777216: [268435456, 262144, 4096, 64, 1], 8388608: [536870912, 134217728, 524288, 131072, 2048, 32], 4194304: [268435456, 67108864, 262144, 65536, 16384, 1024], 2097152: [134217728, 33554432, 131072, 32768, 8192, 512], 1048576: [67108864, 65536, 4096, 256, 16], 524288: [134217728, 8388608, 8192, 128, 2], 262144: [67108864, 16777216, 4194304, 16384, 4096, 64, 1], 131072: [536870912, 33554432, 8388608, 2097152, 8192, 2048, 512, 32], 65536: [268435456, 4194304, 1048576, 4096, 1024, 256, 16], 32768: [134217728, 2097152, 2048, 128, 8], 16384: [67108864, 4194304, 262144, 256, 4], 8192: [33554432, 2097152, 524288, 131072, 512, 128, 2], 4096: [16777216, 1048576, 262144, 65536, 256, 64, 16, 1], 2048: [536870912, 8388608, 131072, 32768, 128, 32, 8], 1024: [268435456, 4194304, 6553

In [231]:
b_move_masks = gen_int_masks(BISHOP_MOVE_MASKS)
print(b_move_masks)
print_bits(b_move_masks)

{536870912: [268435456, 16777216, 8388608, 131072, 2048, 32], 268435456: [536870912, 134217728, 16777216, 8388608, 4194304, 65536, 1024], 134217728: [268435456, 67108864, 8388608, 4194304, 2097152, 524288, 32768], 67108864: [134217728, 33554432, 4194304, 2097152, 1048576, 262144, 16384], 33554432: [67108864, 2097152, 1048576, 131072, 8192, 512], 16777216: [536870912, 268435456, 8388608, 524288, 262144, 4096, 64, 1], 8388608: [536870912, 268435456, 134217728, 16777216, 4194304, 524288, 262144, 131072, 2048, 32], 4194304: [268435456, 134217728, 67108864, 8388608, 2097152, 262144, 131072, 65536, 16384, 1024], 2097152: [134217728, 67108864, 33554432, 4194304, 1048576, 131072, 65536, 32768, 8192, 512], 1048576: [67108864, 33554432, 2097152, 65536, 32768, 4096, 256, 16], 524288: [134217728, 16777216, 8388608, 262144, 16384, 8192, 128, 2], 262144: [67108864, 16777216, 8388608, 4194304, 524288, 131072, 16384, 8192, 4096, 64, 1], 131072: [536870912, 33554432, 8388608, 4194304, 2097152, 262144, 

In [232]:
king_masks = gen_int_masks(KING_MASKS)
print(king_masks)
print_bits(king_masks)

{536870912: [268435456, 16777216, 8388608], 268435456: [536870912, 134217728, 16777216, 8388608, 4194304], 134217728: [268435456, 67108864, 8388608, 4194304, 2097152], 67108864: [134217728, 33554432, 4194304, 2097152, 1048576], 33554432: [67108864, 2097152, 1048576], 16777216: [536870912, 268435456, 8388608, 524288, 262144], 8388608: [536870912, 268435456, 134217728, 16777216, 4194304, 524288, 262144, 131072], 4194304: [268435456, 134217728, 67108864, 8388608, 2097152, 262144, 131072, 65536], 2097152: [134217728, 67108864, 33554432, 4194304, 1048576, 131072, 65536, 32768], 1048576: [67108864, 33554432, 2097152, 65536, 32768], 524288: [16777216, 8388608, 262144, 16384, 8192], 262144: [16777216, 8388608, 4194304, 524288, 131072, 16384, 8192, 4096], 131072: [8388608, 4194304, 2097152, 262144, 65536, 8192, 4096, 2048], 65536: [4194304, 2097152, 1048576, 131072, 32768, 4096, 2048, 1024], 32768: [2097152, 1048576, 65536, 2048, 1024], 16384: [524288, 262144, 8192, 512, 256], 8192: [524288, 26

In [233]:
def gen_complete_masks(cpp_var_name, move_masks):
    result = 'static std::unordered_map<unsigned int, unsigned int> {} = {{\n'.format(cpp_var_name)
    ppromo = (1 << 30)
    if 'pawn' in cpp_var_name:
        for r, row in enumerate(QUEEN_MASKS):
            for c, mask in enumerate(row):
                result += '  {{{}, {}}},\n'.format((1 << (29 - (5 * r + c))) | ppromo, eval('0b{}'.format(''.join([str(b) for b in mask.ravel()]))))
    for r, row in enumerate(masks):
        for c, mask in enumerate(row):
            result += '  {{{}, {}}},\n'.format((1 << (29 - (5 * r + c))), eval('0b{}'.format(''.join([str(b) for b in mask.ravel()]))))
    result += '};\n\n'
    return result

In [234]:
print(gen_complete_masks('white_pawn_masks', W_PAWN_ATTACK_MASKS))

static std::unordered_map<unsigned int, unsigned int> white_pawn_masks = {
  {1610612736, 1066027568},
  {1342177280, 1069884680},
  {1207959552, 1055559812},
  {1140850688, 1047873602},
  {1107296256, 1043506721},
  {1090519040, 838619729},
  {1082130432, 972957992},
  {1077936128, 502748292},
  {1075838976, 267627074},
  {1074790400, 133272881},
  {1074266112, 697295506},
  {1074003968, 365949257},
  {1073872896, 720353956},
  {1073807360, 343907666},
  {1073774592, 171936937},
  {1073758208, 625770260},
  {1073750016, 313425802},
  {1073745920, 156728789},
  {1073743872, 614726890},
  {1073742848, 307362917},
  {1073742336, 589980664},
  {1073742080, 278230012},
  {1073741952, 139115502},
  {1073741888, 86319079},
  {1073741856, 580030435},
  {1073741840, 555307807},
  {1073741832, 277130143},
  {1073741828, 138565087},
  {1073741826, 69806335},
  {1073741825, 51680383},
  {536870912, 536870912},
  {268435456, 268435456},
  {134217728, 134217728},
  {67108864, 67108864},
  {33554432

In [235]:
queen_masks = gen_int_masks(QUEEN_MASKS)
print(queen_masks)
print_bits(queen_masks)

{536870912: [268435456, 134217728, 67108864, 33554432, 16777216, 8388608, 524288, 131072, 16384, 2048, 512, 32, 16], 268435456: [536870912, 134217728, 67108864, 33554432, 16777216, 8388608, 4194304, 262144, 65536, 8192, 1024, 256, 8], 134217728: [536870912, 268435456, 67108864, 33554432, 8388608, 4194304, 2097152, 524288, 131072, 32768, 4096, 128, 4], 67108864: [536870912, 268435456, 134217728, 33554432, 4194304, 2097152, 1048576, 262144, 65536, 16384, 2048, 64, 2], 33554432: [536870912, 268435456, 134217728, 67108864, 2097152, 1048576, 131072, 32768, 8192, 1024, 512, 32, 1], 16777216: [536870912, 268435456, 8388608, 4194304, 2097152, 1048576, 524288, 262144, 16384, 4096, 512, 64, 16, 1], 8388608: [536870912, 268435456, 134217728, 16777216, 4194304, 2097152, 1048576, 524288, 262144, 131072, 8192, 2048, 256, 32, 8], 4194304: [268435456, 134217728, 67108864, 16777216, 8388608, 2097152, 1048576, 262144, 131072, 65536, 16384, 4096, 1024, 128, 4], 2097152: [134217728, 67108864, 33554432, 16

In [236]:
pawn_promotion_masks = {k | (1 << 30): v for k, v in queen_masks.items()}
pawn_promotion_masks

{1073741825: [33554432,
  16777216,
  1048576,
  262144,
  32768,
  4096,
  1024,
  64,
  32,
  16,
  8,
  4,
  2],
 1073741826: [67108864,
  2097152,
  524288,
  65536,
  8192,
  2048,
  128,
  64,
  32,
  16,
  8,
  4,
  1],
 1073741828: [134217728,
  4194304,
  131072,
  16384,
  4096,
  1024,
  256,
  128,
  64,
  16,
  8,
  2,
  1],
 1073741832: [268435456,
  8388608,
  262144,
  32768,
  8192,
  2048,
  512,
  256,
  128,
  16,
  4,
  2,
  1],
 1073741840: [536870912,
  16777216,
  1048576,
  524288,
  65536,
  16384,
  4096,
  512,
  256,
  8,
  4,
  2,
  1],
 1073741856: [536870912,
  33554432,
  8388608,
  1048576,
  131072,
  32768,
  2048,
  1024,
  512,
  256,
  128,
  64,
  2,
  1],
 1073741888: [67108864,
  16777216,
  2097152,
  262144,
  65536,
  4096,
  2048,
  1024,
  512,
  256,
  128,
  32,
  4,
  2,
  1],
 1073741952: [134217728,
  4194304,
  524288,
  131072,
  32768,
  8192,
  4096,
  2048,
  512,
  256,
  64,
  32,
  8,
  4,
  2],
 1073742080: [268435456,
  8388

In [237]:
black_pawn_attack_masks = {**pawn_promotion_masks, **b_pa_masks}
black_pawn_attack_masks

{1: [],
 2: [],
 4: [],
 8: [],
 16: [],
 32: [2],
 64: [4, 1],
 128: [8, 2],
 256: [16, 4],
 512: [8],
 1024: [64],
 2048: [128, 32],
 4096: [256, 64],
 8192: [512, 128],
 16384: [256],
 32768: [2048],
 65536: [4096, 1024],
 131072: [8192, 2048],
 262144: [16384, 4096],
 524288: [8192],
 1048576: [65536],
 2097152: [131072, 32768],
 4194304: [262144, 65536],
 8388608: [524288, 131072],
 16777216: [262144],
 33554432: [2097152],
 67108864: [4194304, 1048576],
 134217728: [8388608, 2097152],
 268435456: [16777216, 4194304],
 536870912: [8388608],
 1073741825: [33554432,
  16777216,
  1048576,
  262144,
  32768,
  4096,
  1024,
  64,
  32,
  16,
  8,
  4,
  2],
 1073741826: [67108864,
  2097152,
  524288,
  65536,
  8192,
  2048,
  128,
  64,
  32,
  16,
  8,
  4,
  1],
 1073741828: [134217728,
  4194304,
  131072,
  16384,
  4096,
  1024,
  256,
  128,
  64,
  16,
  8,
  2,
  1],
 1073741832: [268435456,
  8388608,
  262144,
  32768,
  8192,
  2048,
  512,
  256,
  128,
  16,
  4,
  2,


In [238]:
black_pawn_move_masks = {**pawn_promotion_masks, **b_pm_masks}
black_pawn_move_masks

{1: [],
 2: [],
 4: [],
 8: [],
 16: [],
 32: [1],
 64: [2],
 128: [4],
 256: [8],
 512: [16],
 1024: [32],
 2048: [64],
 4096: [128],
 8192: [256],
 16384: [512],
 32768: [1024],
 65536: [2048],
 131072: [4096],
 262144: [8192],
 524288: [16384],
 1048576: [32768],
 2097152: [65536],
 4194304: [131072],
 8388608: [262144],
 16777216: [524288],
 33554432: [1048576],
 67108864: [2097152],
 134217728: [4194304],
 268435456: [8388608],
 536870912: [16777216],
 1073741825: [33554432,
  16777216,
  1048576,
  262144,
  32768,
  4096,
  1024,
  64,
  32,
  16,
  8,
  4,
  2],
 1073741826: [67108864,
  2097152,
  524288,
  65536,
  8192,
  2048,
  128,
  64,
  32,
  16,
  8,
  4,
  1],
 1073741828: [134217728,
  4194304,
  131072,
  16384,
  4096,
  1024,
  256,
  128,
  64,
  16,
  8,
  2,
  1],
 1073741832: [268435456,
  8388608,
  262144,
  32768,
  8192,
  2048,
  512,
  256,
  128,
  16,
  4,
  2,
  1],
 1073741840: [536870912,
  16777216,
  1048576,
  524288,
  65536,
  16384,
  4096,
 

In [239]:
white_pawn_attack_masks = {**pawn_promotion_masks, ** w_pa_masks}
white_pawn_attack_masks

{1: [64],
 2: [128, 32],
 4: [256, 64],
 8: [512, 128],
 16: [256],
 32: [2048],
 64: [4096, 1024],
 128: [8192, 2048],
 256: [16384, 4096],
 512: [8192],
 1024: [65536],
 2048: [131072, 32768],
 4096: [262144, 65536],
 8192: [524288, 131072],
 16384: [262144],
 32768: [2097152],
 65536: [4194304, 1048576],
 131072: [8388608, 2097152],
 262144: [16777216, 4194304],
 524288: [8388608],
 1048576: [67108864],
 2097152: [134217728, 33554432],
 4194304: [268435456, 67108864],
 8388608: [536870912, 134217728],
 16777216: [268435456],
 33554432: [],
 67108864: [],
 134217728: [],
 268435456: [],
 536870912: [],
 1073741825: [33554432,
  16777216,
  1048576,
  262144,
  32768,
  4096,
  1024,
  64,
  32,
  16,
  8,
  4,
  2],
 1073741826: [67108864,
  2097152,
  524288,
  65536,
  8192,
  2048,
  128,
  64,
  32,
  16,
  8,
  4,
  1],
 1073741828: [134217728,
  4194304,
  131072,
  16384,
  4096,
  1024,
  256,
  128,
  64,
  16,
  8,
  2,
  1],
 1073741832: [268435456,
  8388608,
  262144,
  

In [240]:
white_pawn_move_masks = {**pawn_promotion_masks, **w_pm_masks}
white_pawn_move_masks

{1: [32],
 2: [64],
 4: [128],
 8: [256],
 16: [512],
 32: [1024],
 64: [2048],
 128: [4096],
 256: [8192],
 512: [16384],
 1024: [32768],
 2048: [65536],
 4096: [131072],
 8192: [262144],
 16384: [524288],
 32768: [1048576],
 65536: [2097152],
 131072: [4194304],
 262144: [8388608],
 524288: [16777216],
 1048576: [33554432],
 2097152: [67108864],
 4194304: [134217728],
 8388608: [268435456],
 16777216: [536870912],
 33554432: [],
 67108864: [],
 134217728: [],
 268435456: [],
 536870912: [],
 1073741825: [33554432,
  16777216,
  1048576,
  262144,
  32768,
  4096,
  1024,
  64,
  32,
  16,
  8,
  4,
  2],
 1073741826: [67108864,
  2097152,
  524288,
  65536,
  8192,
  2048,
  128,
  64,
  32,
  16,
  8,
  4,
  1],
 1073741828: [134217728,
  4194304,
  131072,
  16384,
  4096,
  1024,
  256,
  128,
  64,
  16,
  8,
  2,
  1],
 1073741832: [268435456,
  8388608,
  262144,
  32768,
  8192,
  2048,
  512,
  256,
  128,
  16,
  4,
  2,
  1],
 1073741840: [536870912,
  16777216,
  1048576,


In [241]:
rook_masks = gen_int_masks(ROOK_MASKS)
print(rook_masks)
print_bits(rook_masks)

{536870912: [268435456, 134217728, 67108864, 33554432, 16777216, 524288, 16384, 512, 16], 268435456: [536870912, 134217728, 67108864, 33554432, 8388608, 262144, 8192, 256, 8], 134217728: [536870912, 268435456, 67108864, 33554432, 4194304, 131072, 4096, 128, 4], 67108864: [536870912, 268435456, 134217728, 33554432, 2097152, 65536, 2048, 64, 2], 33554432: [536870912, 268435456, 134217728, 67108864, 1048576, 32768, 1024, 32, 1], 16777216: [536870912, 8388608, 4194304, 2097152, 1048576, 524288, 16384, 512, 16], 8388608: [268435456, 16777216, 4194304, 2097152, 1048576, 262144, 8192, 256, 8], 4194304: [134217728, 16777216, 8388608, 2097152, 1048576, 131072, 4096, 128, 4], 2097152: [67108864, 16777216, 8388608, 4194304, 1048576, 65536, 2048, 64, 2], 1048576: [33554432, 16777216, 8388608, 4194304, 2097152, 32768, 1024, 32, 1], 524288: [536870912, 16777216, 262144, 131072, 65536, 32768, 16384, 512, 16], 262144: [268435456, 8388608, 524288, 131072, 65536, 32768, 8192, 256, 8], 131072: [134217728

In [242]:
shadow_seeds = [
    np.array([
        [1,1,1,1,0,1,1,1,1],
        [1,1,1,1,0,1,1,1,1],
        [1,1,1,1,0,1,1,1,1],
        [1,1,1,1,0,1,1,1,1],
        [1,1,1,1,0,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1]
    ]),
    np.array([
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,0],
        [1,1,1,1,1,1,1,0,1],
        [1,1,1,1,1,1,0,1,1],
        [1,1,1,1,1,0,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1]
    ]),
    np.array([
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,0,0,0,0],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1]
    ]),
    np.array([
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,0,1,1,1],
        [1,1,1,1,1,1,0,1,1],
        [1,1,1,1,1,1,1,0,1],
        [1,1,1,1,1,1,1,1,0],
        [1,1,1,1,1,1,1,1,1]
    ]),
    np.array([
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,0,1,1,1,1],
        [1,1,1,1,0,1,1,1,1],
        [1,1,1,1,0,1,1,1,1],
        [1,1,1,1,0,1,1,1,1],
        [1,1,1,1,0,1,1,1,1]
    ]),
    np.array([
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,0,1,1,1,1,1],
        [1,1,0,1,1,1,1,1,1],
        [1,0,1,1,1,1,1,1,1],
        [0,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1]
    ]),
    np.array([
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [0,0,0,0,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1]
    ]),
    np.array([
        [1,1,1,1,1,1,1,1,1],
        [0,1,1,1,1,1,1,1,1],
        [1,0,1,1,1,1,1,1,1],
        [1,1,0,1,1,1,1,1,1],
        [1,1,1,0,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1],
        [1,1,1,1,1,1,1,1,1]
    ])
]

direction_seeds = [np.logical_not(arr).astype(np.int) for arr in shadow_seeds]
for seed in direction_seeds:
    seed[5,4] = 0

In [243]:
shadow_masks = [
    np.rot90(
        stride_tricks.as_strided(
            seed, 
            shape=(6, 5, 6, 5), 
            strides=seed.strides + seed.strides), 
        k=2) for seed in shadow_seeds]
shadow_masks

[array([[[[1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1]],
 
         [[1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1]],
 
         [[1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1]],
 
         [[1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1]],
 
         [[1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1]]],
 
 
        [[[0, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1],
          [1, 1, 1, 1, 1]],
 
       

In [244]:
direction_masks = [
    np.rot90(
        stride_tricks.as_strided(
            seed, 
            shape=(6, 5, 6, 5), 
            strides=seed.strides + seed.strides), 
        k=2) for seed in direction_seeds]
direction_masks

[array([[[[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],
 
         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],
 
         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],
 
         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],
 
         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]]],
 
 
        [[[1, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],
 
       

In [245]:
from collections import defaultdict
shadows = defaultdict(dict)
for d, masks in enumerate(direction_masks):
    print('direction: {}'.format(d))
    print('-'*100)
    for r, row in enumerate(masks):
        for c, d_mask in enumerate(row):
            print('A: {:030b}'.format(1 << 29-(5*r+c)))
            piece_to_move = 1 << 29-(5*r+c)
            ppawn = (1 << 30) | piece_to_move
            m = mask_to_int(d_mask)
            if d == 0:
                for i in range(5, 21, 5):
                    sh_m = m & (m << i) & ((1 << 30) - 1)
                    if sh_m:
                        blocking_piece = piece_to_move << i
                        pp_blocking_piece = (1 << 30) | blocking_piece
                        print('B: {:030b}'.format(blocking_piece))
                        print('S: {:030b}'.format(sh_m))
                        shadows[piece_to_move][blocking_piece] = sh_m
                        shadows[piece_to_move][pp_blocking_piece] = sh_m
                        shadows[ppawn][blocking_piece] = sh_m
                        shadows[ppawn][pp_blocking_piece] = sh_m
                    else:
                        continue
            elif d == 1:
                for i in range(4, 17, 4):
                    sh_m = m & (m << i) & ((1 << 30) - 1)
                    if sh_m:
                        blocking_piece = piece_to_move << i
                        pp_blocking_piece = (1 << 30) | blocking_piece
                        print('B: {:030b}'.format(blocking_piece))
                        print('S: {:030b}'.format(sh_m))
                        shadows[piece_to_move][blocking_piece] = sh_m
                        shadows[piece_to_move][pp_blocking_piece] = sh_m
                        shadows[ppawn][blocking_piece] = sh_m
                        shadows[ppawn][pp_blocking_piece] = sh_m
                    else:
                        continue
            elif d == 2:
                for i in range(1,5):
                    sh_m = m & (m >> i) & ((1 << 30) - 1)
                    if sh_m:
                        blocking_piece = piece_to_move >> i
                        pp_blocking_piece = (1 << 30) | blocking_piece
                        print('B: {:030b}'.format(blocking_piece))
                        print('S: {:030b}'.format(sh_m))
                        shadows[piece_to_move][blocking_piece] = sh_m
                        shadows[piece_to_move][pp_blocking_piece] = sh_m
                        shadows[ppawn][blocking_piece] = sh_m
                        shadows[ppawn][pp_blocking_piece] = sh_m
                    else:
                        continue
            elif d == 3:
                for i in range(6, 25, 6):
                    sh_m = m & (m >> i) & ((1 << 30) - 1)
                    if sh_m:
                        blocking_piece = piece_to_move >> i
                        pp_blocking_piece = (1 << 30) | blocking_piece
                        print('B: {:030b}'.format(blocking_piece))
                        print('S: {:030b}'.format(sh_m))
                        shadows[piece_to_move][blocking_piece] = sh_m
                        shadows[piece_to_move][pp_blocking_piece] = sh_m
                        shadows[ppawn][blocking_piece] = sh_m
                        shadows[ppawn][pp_blocking_piece] = sh_m
                    else:
                        continue
            elif d == 4:
                for i in range(5, 21, 5):
                    sh_m = m & (m >> i) & ((1 << 30) - 1)
                    if sh_m:
                        blocking_piece = piece_to_move >> i
                        pp_blocking_piece = (1 << 30) | blocking_piece
                        print('B: {:030b}'.format(blocking_piece))
                        print('S: {:030b}'.format(sh_m))
                        shadows[piece_to_move][blocking_piece] = sh_m
                        shadows[piece_to_move][pp_blocking_piece] = sh_m
                        shadows[ppawn][blocking_piece] = sh_m
                        shadows[ppawn][pp_blocking_piece] = sh_m
                    else:
                        continue
            elif d == 5:
                for i in range(4, 17, 4):
                    sh_m = m & (m >> i) & ((1 << 30) - 1)
                    if sh_m:
                        blocking_piece = piece_to_move >> i
                        pp_blocking_piece = (1 << 30) | blocking_piece
                        print('B: {:030b}'.format(blocking_piece))
                        print('S: {:030b}'.format(sh_m))
                        shadows[piece_to_move][blocking_piece] = sh_m
                        shadows[piece_to_move][pp_blocking_piece] = sh_m
                        shadows[ppawn][blocking_piece] = sh_m
                        shadows[ppawn][pp_blocking_piece] = sh_m
                    else:
                        continue
            elif d == 6:
                for i in range(1,5):
                    sh_m = m & (m << i) & ((1 << 30) - 1)
                    if sh_m:
                        blocking_piece = piece_to_move << i
                        pp_blocking_piece = (1 << 30) | blocking_piece
                        print('B: {:030b}'.format(blocking_piece))
                        print('S: {:030b}'.format(sh_m))
                        shadows[piece_to_move][blocking_piece] = sh_m
                        shadows[piece_to_move][pp_blocking_piece] = sh_m
                        shadows[ppawn][blocking_piece] = sh_m
                        shadows[ppawn][pp_blocking_piece] = sh_m
                    else:
                        continue
            else:
                for i in range(6, 25, 6):
                    sh_m = m & (m << i) & ((1 << 30) - 1)
                    if sh_m:
                        blocking_piece = piece_to_move << i
                        pp_blocking_piece = (1 << 30) | blocking_piece
                        print('B: {:030b}'.format(blocking_piece))
                        print('S: {:030b}'.format(sh_m))
                        shadows[piece_to_move][blocking_piece] = sh_m
                        shadows[piece_to_move][pp_blocking_piece] = sh_m
                        shadows[ppawn][blocking_piece] = sh_m
                        shadows[ppawn][pp_blocking_piece] = sh_m
                    else:
                        continue

direction: 0
----------------------------------------------------------------------------------------------------
A: 100000000000000000000000000000
A: 010000000000000000000000000000
A: 001000000000000000000000000000
A: 000100000000000000000000000000
A: 000010000000000000000000000000
A: 000001000000000000000000000000
A: 000000100000000000000000000000
A: 000000010000000000000000000000
A: 000000001000000000000000000000
A: 000000000100000000000000000000
A: 000000000010000000000000000000
B: 000001000000000000000000000000
S: 100000000000000000000000000000
A: 000000000001000000000000000000
B: 000000100000000000000000000000
S: 010000000000000000000000000000
A: 000000000000100000000000000000
B: 000000010000000000000000000000
S: 001000000000000000000000000000
A: 000000000000010000000000000000
B: 000000001000000000000000000000
S: 000100000000000000000000000000
A: 000000000000001000000000000000
B: 000000000100000000000000000000
S: 000010000000000000000000000000
A: 000000000000000100000000000000
B:

In [246]:
attack_masks = [king_masks, queen_masks, b_attack_masks, n_masks, rook_masks, b_pa_masks, w_pa_masks]
attacks = {
    'KING_ATTACK_MASKS': king_masks, 
    'QUEEN_ATTACK_MASKS': queen_masks, 
    'BISHOP_ATTACK_MASKS': b_attack_masks, 
    'KNIGHT_ATTACK_MASKS': n_masks, 
    'ROOK_ATTACK_MASKS': rook_masks, 
    'BLACK_PAWN_ATTACK_MASKS': black_pawn_attack_masks, 
    'WHITE_PAWN_ATTACK_MASKS': white_pawn_attack_masks 
}

complete_attacks = {
    'KING_ATTACK_MASKS_COMBINED': KING_MASKS,
    'QUEEN_ATTACK_MASKS_COMBINED': QUEEN_MASKS,
    'BISHOP_ATTACK_MASKS_COMBINED': BISHOP_ATTACK_MASKS,
    'KNIGHT_ATTACK_MASKS_COMBINED': KNIGHT_MASKS,
    'ROOK_ATTACK_MASKS_COMBINED': ROOK_MASKS,
    'BLACK_PAWN_ATTACK_MASKS_COMBINED': B_PAWN_ATTACK_MASKS,
    'WHITE_PAWN_ATTACK_MASKS_COMBINED': W_PAWN_ATTACK_MASKS
}

moves = {
    'KING_MOVE_MASKS': king_masks, 
    'QUEEN_MOVE_MASKS': queen_masks, 
    'BISHOP_MOVE_MASKS': b_move_masks, 
    'KNIGHT_MOVE_MASKS': n_masks, 
    'ROOK_MOVE_MASKS': rook_masks, 
    'BLACK_PAWN_MOVE_MASKS': black_pawn_move_masks, 
    'WHITE_PAWN_MOVE_MASKS': white_pawn_move_masks
}

complete_moves = {
    'KING_MOVE_MASKS_COMBINED': KING_MASKS,
    'QUEEN_MOVE_MASKS_COMBINED': QUEEN_MASKS,
    'BISHOP_MOVE_MASKS_COMBINED': BISHOP_MOVE_MASKS,
    'KNIGHT_MOVE_MASKS_COMBINED': KNIGHT_MASKS,
    'ROOK_MOVE_MASKS_COMBINED': ROOK_MASKS,
    'BLACK_PAWN_MOVE_MASKS_COMBINED': B_PAWN_MOVE_MASKS,
    'WHITE_PAWN_MOVE_MASKS_COMBINED': W_PAWN_MOVE_MASKS
}


black_pawn_vals = np.array([0, 0, 0, 0, 0, 10, 25, 35, 15, 10, 5, 20, 30, 10, 5, 0, 10, 20, -5, -10, 0, -5, -10, 0, 0, 0, 0, 0, 0, 0])
white_pawn_vals = np.flip(black_pawn_vals, axis=0)

header_file = '''//
// Created by Michael Lane on 6/2/17.
// This file requires the use of the --std=c++11 flag when compiling.
//

#ifndef BITBOARD_TABLES_H
#define BITBOARD_TABLES_H

#include <vector>
#include <tuple>
#include <unordered_map>
#include <string>
#include <map>
#include <queue>

#define BLACK_KING 0
#define WHITE_KING 19
#define MOVE_NUMBER 20
#define PLAYER_ON_MOVE 21
#define LOCATION_OF_OPPONENTS 22
#define LOCATION_OF_EMPTY 23
#define TIME_LEFT 24

typedef std::unordered_map<unsigned int, std::vector<unsigned int>> Attack_Masks;
typedef std::unordered_map<unsigned int, std::vector<unsigned int>> Move_Masks;
typedef std::unordered_map<unsigned int, std::string> Int_to_Str;
typedef std::unordered_map<unsigned int, int> Int_to_Idx;
typedef std::unordered_map<unsigned int, std::unordered_map<unsigned int, unsigned int>> Shadow_Mask;
typedef std::vector<unsigned int> State_t;

static unsigned int BLACK_IDX = 0;
static unsigned int WHITE_IDX = 10;

enum player : int {
  BLACK = 1,
  WHITE = 2
};

static Int_to_Idx TO_IDX = {
    {1610612736, 29},
    {1342177280, 28},
    {1207959552, 27},
    {1140850688, 26},
    {1107296256, 25},
    {1090519040, 24},
    {1082130432, 23},
    {1077936128, 22},
    {1075838976, 21},
    {1074790400, 20},
    {1074266112, 19},
    {1074003968, 18},
    {1073872896, 17},
    {1073807360, 16},
    {1073774592, 15},
    {1073758208, 14},
    {1073750016, 13},
    {1073745920, 12},
    {1073743872, 11},
    {1073742848, 10},
    {1073742336, 9},
    {1073742080, 8},
    {1073741952, 7},
    {1073741888, 6},
    {1073741856, 5},
    {1073741840, 4},
    {1073741832, 3},
    {1073741828, 2},
    {1073741826, 1},
    {1073741825, 0},
    {536870912,  29},
    {268435456,  28},
    {134217728,  27},
    {67108864,   26},
    {33554432,   25},
    {16777216,   24},
    {8388608,    23},
    {4194304,    22},
    {2097152,    21},
    {1048576,    20},
    {524288,     19},
    {262144,     18},
    {131072,     17},
    {65536,      16},
    {32768,      15},
    {16384,      14},
    {8192,       13},
    {4096,       12},
    {2048,       11},
    {1024,       10},
    {512,        9},
    {256,        8},
    {128,        7},
    {64,         6},
    {32,         5},
    {16,         4},
    {8,          3},
    {4,          2},
    {2,          1},
    {1,          0},
};

static Int_to_Str TO_STR = {
    {1610612736, "a6"},
    {1342177280, "b6"},
    {1207959552, "c6"},
    {1140850688, "d6"},
    {1107296256, "e6"},
    {1090519040, "a5"},
    {1082130432, "b5"},
    {1077936128, "c5"},
    {1075838976, "d5"},
    {1074790400, "e5"},
    {1074266112, "a4"},
    {1074003968, "b4"},
    {1073872896, "c4"},
    {1073807360, "d4"},
    {1073774592, "e4"},
    {1073758208, "a3"},
    {1073750016, "b3"},
    {1073745920, "c3"},
    {1073743872, "d3"},
    {1073742848, "e3"},
    {1073742336, "a2"},
    {1073742080, "b2"},
    {1073741952, "c2"},
    {1073741888, "d2"},
    {1073741856, "e2"},
    {1073741840, "a1"},
    {1073741832, "b1"},
    {1073741828, "c1"},
    {1073741826, "d1"},
    {1073741825, "e1"},
    {536870912,  "a6"},
    {268435456,  "b6"},
    {134217728,  "c6"},
    {67108864,   "d6"},
    {33554432,   "e6"},
    {16777216,   "a5"},
    {8388608,    "b5"},
    {4194304,    "c5"},
    {2097152,    "d5"},
    {1048576,    "e5"},
    {524288,     "a4"},
    {262144,     "b4"},
    {131072,     "c4"},
    {65536,      "d4"},
    {32768,      "e4"},
    {16384,      "a3"},
    {8192,       "b3"},
    {4096,       "c3"},
    {2048,       "d3"},
    {1024,       "e3"},
    {512,        "a2"},
    {256,        "b2"},
    {128,        "c2"},
    {64,         "d2"},
    {32,         "e2"},
    {16,         "a1"},
    {8,          "b1"},
    {4,          "c1"},
    {2,          "d1"},
    {1,          "e1"},
};

'''

for name, masks in attacks.items():
    header_file += 'static Attack_Masks {n} = {{\n'.format(n=name)
    for k, v in sorted(masks.items(), reverse=True):
        if v:
            header_file += '  {{{k}, {{{v}}}}},\n'.format(k=k, v=str(v)[1:-1])
    header_file += '};\n\n'

for name, masks in complete_attacks.items():
    header_file += gen_complete_masks(name, masks)
    
for name, masks in moves.items():
    header_file += 'static Move_Masks {n} = {{\n'.format(n=name)
    for k, v in sorted(masks.items(), reverse=True):
        if v:
            header_file += '  {{{k}, {{{v}}}}},\n'.format(k=k, v=str(v)[1:-1])
    header_file += '};\n\n'

for name, masks in complete_moves.items():
    header_file += gen_complete_masks(name, masks)
    
header_file += 'static Shadow_Mask SHADOW_MASK = {\n'
for k in sorted(shadows.keys(), reverse=True):
    header_file += '  {\n'
    header_file += '    {}, {{\n'.format(k)
    for vk, v in sorted(shadows[k].items(), reverse=True):
        header_file += '    {{{}, {}}},\n'.format(vk, v)
    header_file += '    }\n'
    header_file += '  },\n'
header_file += '};\n\n'

begin_wq_heuristic = '''1000 -500 -500 -500 -500
-500 -500 -500 -500 -500
-500 -500 -500 -500 -500
-500 -500 -500 -500 -500
-500 -500 -500 -500    0
-500 -500 -500    0 -500'''

begin_white_queen_heuristic, begin_black_queen_heuristic = raw_heuristic_to_b_w_cpp('begin_white_queen_heuristic', 'begin_black_queen_heuristic', begin_wq_heuristic)

header_file += begin_white_queen_heuristic
header_file += begin_black_queen_heuristic

mid_wq_heuristic = '''-20 -10  -5 -10 -20
-10   0   0   0 -10
-10   5   5   5 -10
  0   5   5   5   0
-10   5   5   5 100
-20 -10  -5 -10 -20'''

white_queen_heuristic, black_queen_heuristic = raw_heuristic_to_b_w_cpp('white_queen_heuristic', 'black_queen_heuristic', mid_wq_heuristic)

header_file += white_queen_heuristic
header_file += black_queen_heuristic

wp_heuristic = ''' 1000 1000 1000 1000 1000
10  25  35  15  10
 5  20  30  10   5
 0  10  20  -5   0
 0  -5 -10   0 -10
 0   0   0   0   0'''
white_pawn_heuristic, black_pawn_heuristic = raw_heuristic_to_b_w_cpp('white_pawn_heuristic', 'black_pawn_heuristic', wp_heuristic, wq_heuristic)

header_file += white_pawn_heuristic
header_file += black_pawn_heuristic

begin_white_knight_heuristic = ''' 500 0 0 0 0
    0   0    0 0 0 
    0   0    0 0 0
 -500   0 -500 0 0
    0   0    0 0 0 
    0 300    0 0 0'''

begin_white_knight_heuristic, begin_black_knight_heuristic = raw_heuristic_to_b_w_cpp('begin_white_knight_heuristic', 'begin_black_knight_heuristic', begin_white_knight_heuristic)

header_file += begin_white_knight_heuristic
header_file += begin_black_knight_heuristic

wn_heuristic = ''' -50 -40 -30 -40 -50
 -40   5  10   5 -40 
 -30  10  20  10 -30
 -30  10  20  10 -30
 -40   5  10   5 -40 
 -50 -40 -30 -40 -50'''
white_knight_heuristic, black_knight_heuristic = raw_heuristic_to_b_w_cpp('middle_white_knight_heuristic', 'middle_black_knight_heuristic', wn_heuristic)

header_file += white_knight_heuristic
header_file += black_knight_heuristic

wb_heuristic = '''-20 -15 -10 -15 -20
-15   5   5   5 -15
-10   5  15   5 -10
-15  10  10  10 -15
-10   0  10   0 -10
-25 -10 -15 -10 -25'''

white_bishop_heuristic, black_bishop_heuristic = raw_heuristic_to_b_w_cpp('white_bishop_heuristic', 'black_bishop_heuristic', wb_heuristic)

header_file += white_bishop_heuristic
header_file += black_bishop_heuristic

wr_heuristic = '''  0   0   5   0   0  
-10   0   0   0 -10
-10   0   0   0 -10
-10   0   0   0 -10  
  5  10  10  10   5
  0   0   0   0   0'''

white_rook_heuristic, black_rook_heuristic = raw_heuristic_to_b_w_cpp('white_rook_heuristic', 'black_rook_heuristic', wr_heuristic)

header_file += white_rook_heuristic
header_file += black_rook_heuristic

wk_b_heuristic = '''-500 -500 -500 -500 -500
-500 -500 -500 -500 -500
-500 -500 -500 -500 -500
-500 -500 -500 -500 -500
-500 -500 -500 -500 -500
-500 -500 -500 -500    0'''

white_king_begin_game_heuristic, black_king_begin_game_heuristic = raw_heuristic_to_b_w_cpp('white_king_begin_game_heuristic', 'black_king_begin_game_heuristic', wk_b_heuristic)

header_file += white_king_begin_game_heuristic
header_file += black_king_begin_game_heuristic

wk_m_heuristic = '''-30 -40 -50 -40 -30
-30 -40 -50 -40 -30
-30 -40 -50 -40 -30
-20 -30 -40 -30 -20
-10 -20 -30 -20 -10
 50  40   0  40  50'''

white_king_middle_game_heuristic, black_king_middle_game_heuristic = raw_heuristic_to_b_w_cpp('white_king_middle_game_heuristic', 'black_king_middle_game_heuristic', wk_m_heuristic)

header_file += white_king_middle_game_heuristic
header_file += black_king_middle_game_heuristic

wk_e_heuristic = '''-50 -40 -30 -40 -50
-30 -15   0 -15 -30
-30  15  40  15 -30
-30  15  40  15 -30
-30   0   0   0 -30
-50 -40 -30 -40 -50'''

white_king_end_game_heuristic, black_king_end_game_heuristic = raw_heuristic_to_b_w_cpp('white_king_end_game_heuristic', 'black_king_end_game_heuristic', wk_e_heuristic)

header_file += white_king_end_game_heuristic
header_file += black_king_end_game_heuristic

header_file += '''static Attack_Masks type_to_attack[20] = {
    KING_ATTACK_MASKS,
    QUEEN_ATTACK_MASKS,
    BISHOP_ATTACK_MASKS,
    KNIGHT_ATTACK_MASKS,
    ROOK_ATTACK_MASKS,
    BLACK_PAWN_ATTACK_MASKS,
    BLACK_PAWN_ATTACK_MASKS,
    BLACK_PAWN_ATTACK_MASKS,
    BLACK_PAWN_ATTACK_MASKS,
    BLACK_PAWN_ATTACK_MASKS,
    WHITE_PAWN_ATTACK_MASKS,
    WHITE_PAWN_ATTACK_MASKS,
    WHITE_PAWN_ATTACK_MASKS,
    WHITE_PAWN_ATTACK_MASKS,
    WHITE_PAWN_ATTACK_MASKS,
    ROOK_ATTACK_MASKS,
    KNIGHT_ATTACK_MASKS,
    BISHOP_ATTACK_MASKS,
    QUEEN_ATTACK_MASKS,
    KING_ATTACK_MASKS
};

static Move_Masks type_to_move[20] = {
    KING_MOVE_MASKS,
    QUEEN_MOVE_MASKS,
    BISHOP_MOVE_MASKS,
    KNIGHT_MOVE_MASKS,
    ROOK_MOVE_MASKS,
    BLACK_PAWN_MOVE_MASKS,
    BLACK_PAWN_MOVE_MASKS,
    BLACK_PAWN_MOVE_MASKS,
    BLACK_PAWN_MOVE_MASKS,
    BLACK_PAWN_MOVE_MASKS,
    WHITE_PAWN_MOVE_MASKS,
    WHITE_PAWN_MOVE_MASKS,
    WHITE_PAWN_MOVE_MASKS,
    WHITE_PAWN_MOVE_MASKS,
    WHITE_PAWN_MOVE_MASKS,
    ROOK_MOVE_MASKS,
    KNIGHT_MOVE_MASKS,
    BISHOP_MOVE_MASKS,
    QUEEN_MOVE_MASKS,
    KING_MOVE_MASKS
};

static const int black_on_move_values[20]{
    10000,  // MY KING
    900,    // MY QUEEN
    300,    // MY BISHOP
    300,    // MY KNIGHT
    500,    // MY ROOK
    100,    // MY PAWN
    100,    // MY PAWN
    100,    // MY PAWN
    100,    // MY PAWN
    100,    // MY PAWN
    -100,   // OPPONENT PAWN
    -100,   // OPPONENT PAWN
    -100,   // OPPONENT PAWN
    -100,   // OPPONENT PAWN
    -100,   // OPPONENT PAWN
    -500,   // OPPONENT ROOK
    -300,   // OPPONENT KNIGHT
    -300,   // OPPONENT BISHOP
    -900,   // OPPONENT QUEEN
    -10000  // OPPONENT KING
};

static const int white_on_move_values[20]{
    -10000,  // OPPONENT KING
    -900,    // OPPONENT QUEEN
    -300,    // OPPONENT BISHOP
    -300,    // OPPONENT KNIGHT
    -500,    // OPPONENT ROOK
    -100,    // OPPONENT PAWN
    -100,    // OPPONENT PAWN
    -100,    // OPPONENT PAWN
    -100,    // OPPONENT PAWN
    -100,    // OPPONENT PAWN
    100,   // MY PAWN
    100,   // MY PAWN
    100,   // MY PAWN
    100,   // MY PAWN
    100,   // MY PAWN
    500,   // MY ROOK
    300,   // MY KNIGHT
    300,  // MY BISHOP
    900,  // MY QUEEN
    10000  // MY KING
};

static const int piece_type_zobrist_index[20]{
    0,  // k
    1,  // q
    2,  // b
    3,  // n
    4,  // r
    5,  // p
    5,  // p
    5,  // p
    5,  // p
    5,  // p
    6,  // P
    6,  // P
    6,  // P
    6,  // P
    6,  // P
    7,  // R
    8,  // N
    9,  // B
    10, // Q
    11, // K
};

static std::unordered_map<unsigned int, int> begin_heuristic_dispatch[20] = {
    black_king_begin_game_heuristic,
    begin_black_queen_heuristic,
    black_bishop_heuristic,
    begin_black_knight_heuristic,
    black_rook_heuristic,
    black_pawn_heuristic,
    black_pawn_heuristic,
    black_pawn_heuristic,
    black_pawn_heuristic,
    black_pawn_heuristic,
    white_pawn_heuristic,
    white_pawn_heuristic,
    white_pawn_heuristic,
    white_pawn_heuristic,
    white_pawn_heuristic,
    white_rook_heuristic,
    begin_white_knight_heuristic,
    white_bishop_heuristic,
    begin_white_queen_heuristic,
    white_king_begin_game_heuristic,
};

static std::unordered_map<unsigned int, int> middle_heuristic_dispatch[20] = {
    black_king_middle_game_heuristic,
    black_queen_heuristic,
    black_bishop_heuristic,
    middle_black_knight_heuristic,
    black_rook_heuristic,
    black_pawn_heuristic,
    black_pawn_heuristic,
    black_pawn_heuristic,
    black_pawn_heuristic,
    black_pawn_heuristic,
    white_pawn_heuristic,
    white_pawn_heuristic,
    white_pawn_heuristic,
    white_pawn_heuristic,
    white_pawn_heuristic,
    white_rook_heuristic,
    middle_white_knight_heuristic,
    white_bishop_heuristic,
    white_queen_heuristic,
    white_king_middle_game_heuristic,
};

static std::unordered_map<unsigned int, int> end_heuristic_dispatch[20] = {
    black_king_end_game_heuristic,
    black_queen_heuristic,
    black_bishop_heuristic,
    middle_black_knight_heuristic,
    black_rook_heuristic,
    black_pawn_heuristic,
    black_pawn_heuristic,
    black_pawn_heuristic,
    black_pawn_heuristic,
    black_pawn_heuristic,
    white_pawn_heuristic,
    white_pawn_heuristic,
    white_pawn_heuristic,
    white_pawn_heuristic,
    white_pawn_heuristic,
    white_rook_heuristic,
    middle_white_knight_heuristic,
    white_bishop_heuristic,
    white_queen_heuristic,
    white_king_end_game_heuristic,
};

static std::unordered_map<unsigned int, unsigned int> combined_attack_dispatch[20] = {
    KING_ATTACK_MASKS_COMBINED,
    QUEEN_ATTACK_MASKS_COMBINED,
    BISHOP_ATTACK_MASKS_COMBINED,
    KNIGHT_ATTACK_MASKS_COMBINED,
    ROOK_ATTACK_MASKS_COMBINED,
    BLACK_PAWN_ATTACK_MASKS_COMBINED,
    BLACK_PAWN_ATTACK_MASKS_COMBINED,
    BLACK_PAWN_ATTACK_MASKS_COMBINED,
    BLACK_PAWN_ATTACK_MASKS_COMBINED,
    BLACK_PAWN_ATTACK_MASKS_COMBINED,
    WHITE_PAWN_ATTACK_MASKS_COMBINED,
    WHITE_PAWN_ATTACK_MASKS_COMBINED,
    WHITE_PAWN_ATTACK_MASKS_COMBINED,
    WHITE_PAWN_ATTACK_MASKS_COMBINED,
    WHITE_PAWN_ATTACK_MASKS_COMBINED,
    ROOK_ATTACK_MASKS_COMBINED,
    KNIGHT_ATTACK_MASKS_COMBINED,
    BISHOP_ATTACK_MASKS_COMBINED,
    QUEEN_ATTACK_MASKS_COMBINED,
    KING_ATTACK_MASKS_COMBINED
};

static std::unordered_map<unsigned int, unsigned int> combined_move_dispatch[20] = {
    KING_MOVE_MASKS_COMBINED,
    QUEEN_MOVE_MASKS_COMBINED,
    BISHOP_MOVE_MASKS_COMBINED,
    KNIGHT_MOVE_MASKS_COMBINED,
    ROOK_MOVE_MASKS_COMBINED,
    BLACK_PAWN_MOVE_MASKS_COMBINED,
    BLACK_PAWN_MOVE_MASKS_COMBINED,
    BLACK_PAWN_MOVE_MASKS_COMBINED,
    BLACK_PAWN_MOVE_MASKS_COMBINED,
    BLACK_PAWN_MOVE_MASKS_COMBINED,
    WHITE_PAWN_MOVE_MASKS_COMBINED,
    WHITE_PAWN_MOVE_MASKS_COMBINED,
    WHITE_PAWN_MOVE_MASKS_COMBINED,
    WHITE_PAWN_MOVE_MASKS_COMBINED,
    WHITE_PAWN_MOVE_MASKS_COMBINED,
    ROOK_MOVE_MASKS_COMBINED,
    KNIGHT_MOVE_MASKS_COMBINED,
    BISHOP_MOVE_MASKS_COMBINED,
    QUEEN_MOVE_MASKS_COMBINED,
    KING_MOVE_MASKS_COMBINED
};

static std::unordered_map<unsigned int, unsigned int> my_player_index{
    {1, 10},
    {2, 0}
};

static std::unordered_map<unsigned int, unsigned int> opponent_player_index{
    {1, 0},
    {2, 10}
};

static std::unordered_map<unsigned int, unsigned int> opponent{
    {1, 2},
    {2, 1}
};

'''

header_file += '#endif  // BITBOARD_TABLES_H'
print(header_file)



//
// Created by Michael Lane on 6/2/17.
// This file requires the use of the --std=c++11 flag when compiling.
//

#ifndef BITBOARD_TABLES_H
#define BITBOARD_TABLES_H

#include <vector>
#include <tuple>
#include <unordered_map>
#include <string>
#include <map>
#include <queue>

#define BLACK_KING 0
#define WHITE_KING 19
#define MOVE_NUMBER 20
#define PLAYER_ON_MOVE 21
#define LOCATION_OF_OPPONENTS 22
#define LOCATION_OF_EMPTY 23
#define TIME_LEFT 24

typedef std::unordered_map<unsigned int, std::vector<unsigned int>> Attack_Masks;
typedef std::unordered_map<unsigned int, std::vector<unsigned int>> Move_Masks;
typedef std::unordered_map<unsigned int, std::string> Int_to_Str;
typedef std::unordered_map<unsigned int, int> Int_to_Idx;
typedef std::unordered_map<unsigned int, std::unordered_map<unsigned int, unsigned int>> Shadow_Mask;
typedef std::vector<unsigned int> State_t;

static unsigned int BLACK_IDX = 0;
static unsigned int WHITE_IDX = 10;

enum player : int {
  BLACK = 1,
  WHITE 